# Select cells from database

This notebook is intended to retrieve single-cells from a database file.

Requirements:
- Nuclei, Cells and Cytoplasm objects named in that way;
- Output database .db file from ExportToDatabase (CellProfiler);
- Each database file is named after the Metadata_Plate name, e.g. 220607_092050_Plate_1;
    - Provide a plate_list with all plate names: `plate_list = ['plate1', 'plate2', ...]`
- All databased files must be organized such:
    ```
    |- backend 
    |   |- <project_name>
    |   |   |- <plate_name>
    |   |   |   |- <plate1.db>
    |   |   |- <plate_name>
    |   |   |   |- <plate2.db>
    ...
    ```


# Import libraries

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.cluster import KMeans
from openpyxl import Workbook, load_workbook
import os
import easygui as eg
from numpy import random
import sqlite3
import pycytominer
from pycytominer.cyto_utils.cells import SingleCells

## Import database file

### Import every plate and then join them

In [2]:
plate_list = ['220607_092050_Plate_1', '220608_152238_Plate_1', '220617_090443_Plate_1']

- Common path will be sqlite:/// + the location of the directory on your computer (until <project_name>)

In [3]:
common_path = f"sqlite:///D:/2022_09_09_LiveCellPainting_fossa_Cimini/workspace/backend/2022_06_07_DILI/"

- Run the next cell, and wait. This could take a long time (10 minutes or more) depending on the size of the database file.

In [4]:
df_list = []
for plate in plate_list:
    database = SingleCells(common_path + plate + "/" + plate + ".sqlite")
    df_temp = database.merge_single_cells()
    df_list.append(df_temp)

- The next cell will join the plates into a single file containing single-cell information.

In [7]:
df = pd.concat(df_list, axis='index')

In [8]:
df.head()

,Metadata_ImageNumber,Metadata_Plate,Metadata_Site,Metadata_Well,Metadata_TableNumber,Metadata_ObjectNumber_cytoplasm,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_BoundingBoxMaximum_X,Cytoplasm_AreaShape_BoundingBoxMaximum_Y,...,Nuclei_Texture_Variance_CorrPI_10_02_256,Nuclei_Texture_Variance_CorrPI_10_03_256,Nuclei_Texture_Variance_CorrPI_20_00_256,Nuclei_Texture_Variance_CorrPI_20_01_256,Nuclei_Texture_Variance_CorrPI_20_02_256,Nuclei_Texture_Variance_CorrPI_20_03_256,Nuclei_Texture_Variance_CorrPI_5_00_256,Nuclei_Texture_Variance_CorrPI_5_01_256,Nuclei_Texture_Variance_CorrPI_5_02_256,Nuclei_Texture_Variance_CorrPI_5_03_256
0,1,220607_092050_Plate_1,1,B10,1571898871,1,17563,28700,1057,195,...,23.387480,23.456552,24.893570,21.909327,25.605559,26.443746,23.180844,23.307391,22.693203,22.563807
1,1,220607_092050_Plate_1,1,B10,1571898871,2,26625,47064,544,307,...,27.535579,28.164301,29.948233,32.728835,29.760620,28.631319,26.867608,27.496918,26.937428,27.227470
2,1,220607_092050_Plate_1,1,B10,1571898871,3,10090,20703,722,285,...,190.739156,184.776425,193.994949,288.495965,229.479026,227.778470,226.038468,191.521848,187.765931,190.227640
3,1,220607_092050_Plate_1,1,B10,1571898871,4,20770,39366,1068,353,...,30.832619,32.460216,30.510909,30.444835,31.034468,29.681880,31.185042,31.649853,30.655306,30.647967
4,1,220607_092050_Plate_1,1,B10,1571898871,5,27108,59052,409,405,...,111.594226,111.980760,116.428891,115.604023,116.062754,119.931474,115.106248,109.352773,108.304546,108.767129


In [9]:
df.tail()

,Metadata_ImageNumber,Metadata_Plate,Metadata_Site,Metadata_Well,Metadata_TableNumber,Metadata_ObjectNumber_cytoplasm,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_BoundingBoxArea,Cytoplasm_AreaShape_BoundingBoxMaximum_X,Cytoplasm_AreaShape_BoundingBoxMaximum_Y,...,Nuclei_Texture_Variance_CorrPI_10_02_256,Nuclei_Texture_Variance_CorrPI_10_03_256,Nuclei_Texture_Variance_CorrPI_20_00_256,Nuclei_Texture_Variance_CorrPI_20_01_256,Nuclei_Texture_Variance_CorrPI_20_02_256,Nuclei_Texture_Variance_CorrPI_20_03_256,Nuclei_Texture_Variance_CorrPI_5_00_256,Nuclei_Texture_Variance_CorrPI_5_01_256,Nuclei_Texture_Variance_CorrPI_5_02_256,Nuclei_Texture_Variance_CorrPI_5_03_256
50731,1500,220617_090443_Plate_1,9,G9,1328917141,38,5224,13624,135,725,...,38.409776,48.572318,39.223358,48.563899,35.771190,18.207022,39.073237,36.437932,41.391098,43.175009
50732,1500,220617_090443_Plate_1,9,G9,1328917141,39,17801,41067,625,800,...,45.798824,48.543215,48.957625,49.137564,52.250616,57.022030,44.592727,48.394613,44.837983,44.014224
50733,1500,220617_090443_Plate_1,9,G9,1328917141,40,10583,22010,846,813,...,29.725966,30.038200,31.356563,34.854567,30.902707,23.660524,28.144433,28.029887,28.001454,29.398238
50734,1500,220617_090443_Plate_1,9,G9,1328917141,41,9918,23562,1149,790,...,82.840688,84.015090,96.092937,108.861319,96.301985,50.730780,79.653462,80.546822,80.050448,90.831335
50735,1500,220617_090443_Plate_1,9,G9,1328917141,42,11126,24603,761,858,...,57.551067,56.973819,78.305462,101.579515,63.712774,33.791654,56.287266,57.138796,57.691115,72.876791


# Export

In [ ]:
output_path = eg.diropenbox(msg="Choose an output folder", default=r"D:")
print('Path to save the single cell file', output_path)
project_name = input('Provide project name: ')

Path to save the profile D:\2022_09_09_LiveCellPainting_fossa_Cimini\workspace\profiles\2022_06_07_DILI


In [ ]:
df.to_csv(output_path + r'/' + project_name + 'single_cells.csv')
print('Successfully exported to:', output_path + r'/' + project_name + 'single_cells.csv')

Successfully exported to: D:\2022_09_09_LiveCellPainting_fossa_Cimini\workspace\profiles\2022_06_07_DILI/2022_06_07_DILI_normalized_negcon_feature_select.csv


# Stop

I have stopped here because my files are not that big, so my laptop can handle using pycytominer to this operation.

If you have bigger database files and want to retrive specific images/wells, use the code below for that.

## Retrieve specific wells

In [2]:
conn = sqlite3.connect(r"D:\2022_09_09_LiveCellPainting_fossa_Cimini\workspace\backend\2021_10_08_AgNPViability\211020_085918_Plate_1\211020_085918_Plate_1.sqlite")

In [10]:
well_name = ["B2", "B10"]

channels = ["CorrGFP", "CorrPI"]

## Retrieve image numbers from well's names

In [9]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res.fetchall():
    print(name[0])

Image
Nuclei
Object relationships
Cytoplasm
Nucleoli
Vesicles
Cells


In [11]:
#get the names of each well as a list from well_name
rand_img_num=well_name.copy()

list_str="('"
for i in rand_img_num:
    list_str=list_str+str(i)+"','" 
list_str=list_str[:-2]+")"

print(list_str)

('B2','B10')


In [12]:
#define the name of the well
meta_well_col_str="Image_Metadata_Well"

In [13]:
#create a query with the compartment names to be able to retrieve them from the table
compartment_query = "select * from {} WHERE {} IN {};".\
format("Image",meta_well_col_str,list_str)

In [14]:
#select the metadata and image measurements from the Image table belonging to the wells you want
plateImageDf = pd.read_sql(sql=compartment_query, con=conn)

DatabaseError: Execution failed on sql 'select * from Image WHERE Image_Metadata_Well IN ('B2','B10');': no such column: Image_Metadata_Well

In [10]:
#get the ImageNumbers for the wells you want
img_nums=plateImageDf.ImageNumber.unique().tolist()

list_str2="("
for i in img_nums:
    list_str2=list_str2+str(i)+',' 
list_str2=list_str2[:-1]+")"

print(list_str2)

(10,11,12,13,14,15,16,17,18,55,56,57,58,59,60,61,62,63,271,272,273,274,275,276,277,278,279,810,811,812,813,814,815,816,817,818,828,829,830,831,832,833,834,835,836,1620,1621,1622,1623,1624,1625,1626,1627,1628,2106,2107,2108,2109,2110,2111,2112,2113,2114,2538,2539,2540,2541,2542,2543,2544,2545,2546)


In [11]:
#from the compartment names, open each table and retrieve information from the images on list_str2
plateDf_list=[]
compartments=["cells", "cytoplasm", "nuclei"]

for compartment in compartments:
    compartment_query = "select * from {} WHERE {} IN {};".format(compartment,"ImageNumber",list_str2)
    plateDf_list.append(pd.read_sql(sql=compartment_query, con=conn))

In [14]:
#create the dataframe from the Df_list 
plateDf = reduce(lambda left,right: pd.merge(left,right,on=["TableNumber", "ImageNumber", "ObjectNumber"]), plateDf_list)

In [15]:
#merge information from Image table with cells data
plateDfwMeta = pd.merge(plateDf, plateImageDf, on=["TableNumber", "ImageNumber"])

In [16]:
#exclude duplicated cols
plateDfwMeta = plateDfwMeta.loc[:,~plateDfwMeta.columns.duplicated()]

In [17]:
#create a copy
df_p_s0 = plateDfwMeta

In [18]:
#take a look at the dataframe
print(df_p_s0.shape)
df_p_s0.head()

(7577, 2442)


,TableNumber,ImageNumber,ObjectNumber,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,...,Image_Width_IllumAGP,Image_Width_IllumDNA,Image_Width_IllumER,Image_Width_IllumMito,Image_Width_IllumRNA,Image_Width_OrigAGP,Image_Width_OrigDNA,Image_Width_OrigER,Image_Width_OrigMito,Image_Width_OrigRNA
0,1240fc906b760746bfaa4913fdb947b6,10,1,7334,642.0,0.0,1.196197,0.732293,1.0,0.622053,...,2160,2160,2160,2160,2160,2160,2160,2160,2160,2160
1,1240fc906b760746bfaa4913fdb947b6,10,2,7084,1952.0,1.0,1.288208,0.865021,1.0,0.572722,...,2160,2160,2160,2160,2160,2160,2160,2160,2160,2160
2,1240fc906b760746bfaa4913fdb947b6,10,3,13696,2159.0,40.0,1.116869,0.632134,1.0,0.689766,...,2160,2160,2160,2160,2160,2160,2160,2160,2160,2160
3,1240fc906b760746bfaa4913fdb947b6,10,4,9814,1123.0,36.0,1.147676,0.773804,1.0,0.702606,...,2160,2160,2160,2160,2160,2160,2160,2160,2160,2160
4,1240fc906b760746bfaa4913fdb947b6,10,5,16297,1672.0,78.0,1.071092,0.708822,1.0,0.691606,...,2160,2160,2160,2160,2160,2160,2160,2160,2160,2160


## Save selected DataFrame to csv

In [19]:
import pickle
df_p_s0.to_csv("SQ00015195_raw.csv")